In [24]:
import numpy as np
import pandas as pd
import re
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [104]:
# 規格最小值最大值放進來
lcl = 0
ucl = 30

# 從 excel 將原始數據直接複製貼上, 不用額外處理
input_str = """
41.65	41.27	44.56	40.85	40.25	40.62	40.07	43.2	43.3	41.14
41.67	41.26	44.61	40.84	40.28	40.52	39.26	43.23	43.23	41.18
41.67	41.25	44.57	40.85	40.25	40.65	40.18	43.22	43.23	41.19
41.63	41.23	44.56	40.87	40.27	40.54	40.12	43.25	43.19	41.14
41.63	41.2	44.49	40.8	40.31	40.54	40.07	43.19	43.2	41.15
41.62	41.22	44.53	40.79	40.26	40.45	40.05	43.2	43.25	41.12
41.63	41.2	44.54	40.84	40.24	40.54	39.23	43.18	43.28	41.17
41.6	41.21	44.56	40.8	40.21	40.49	40.05	43.24	43.21	41.16
41.64	41.2	44.55	40.8	40.26	40.51	39.99	43.24	43.21	41.13
"""

In [105]:
# 前面修改完後執行該格看結果

tolerance = ucl - lcl

def parse_squashed_floats(input_str):
    data = []
    float_pattern = re.compile(r'-?\d+(?:\.\d+)?')  # Match float numbers
    for line in input_str.strip().split('\n'):
        floats = list(map(float, float_pattern.findall(line)))
        data.append(floats)
    return data
    
input_list = parse_squashed_floats(input_str)
raw_data = np.array(input_list)

n_trials = 3
operator_ids = list(range(1, 1+len(raw_data)//n_trials))
part_ids = list(range(1, 1+len(raw_data[0])))
records = []

for op_index in range(len(operator_ids)):
    for trial in range(n_trials):
        row = raw_data[op_index * n_trials + trial]
        for part_idx, measurement in enumerate(row):
            records.append({
                'Operator': operator_ids[op_index],
                'Trial': trial + 1,
                'Part': f'Part{part_idx+1}',
                'Measurement': measurement
            })

df = pd.DataFrame(records)

# Perform Two-Way ANOVA without interaction
model = ols('Measurement ~ C(Part) + C(Operator)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=1)
anova_table['mean_sq'] = anova_table['sum_sq'] / anova_table['df']

# Extract Mean Squares
MS_part = anova_table.loc['C(Part)', 'mean_sq']
MS_operator = anova_table.loc['C(Operator)', 'mean_sq']
MS_repeat = anova_table.loc['Residual', 'mean_sq']

n_operators = df['Operator'].nunique()
n_parts = df['Part'].nunique()

# Variance Components
var_repeat = MS_repeat
var_operator = max((MS_operator - MS_repeat) / (n_parts * n_trials), 0)
var_part = max((MS_part - MS_repeat) / (n_operators * n_trials), 0)
var_grr = var_repeat + var_operator
var_total = var_repeat + var_operator + var_part

# Standard deviations
sd_repeat = np.sqrt(var_repeat)
sd_operator = np.sqrt(var_operator)
sd_part = np.sqrt(var_part)
sd_grr = np.sqrt(var_grr)
sd_total = np.sqrt(var_total)

results = {
    'Source (來源)': ['Total Gage R&R', 'Repeatability', 'Reproducibility', 'Part-to-Part', 'Total Variation'],
    'SD (標準差)': [sd_grr, sd_repeat, sd_operator, sd_part, sd_total],
    '%Study Variation (%研究變異)': [6 * sd_grr, 6 * sd_repeat, 6 * sd_operator, 6 * sd_part, 6 * sd_total] / (6 * sd_total) * 100,
    '%Tolerance (%公差)': [
       100 * 6 * sd_grr / tolerance,
       100 * 6 * sd_repeat / tolerance,
       100 * 6 * sd_operator / tolerance,
       100 * 6 * sd_part / tolerance,
       100 * 6 * sd_total / tolerance]
}

result_df = pd.DataFrame(results)
pd.set_option('display.float_format', '{:.4f}'.format)
print("\nGage R&R Summary:")
print(result_df)

# NDC = 1.41 × (Part SD / GRR SD)
ndc = 1.41 * sd_part / sd_grr
print(f"\n可區分的類別數 (NDC): {ndc:.2f}, 向下取整數 {int(ndc)}.")


Gage R&R Summary:
       Source (來源)  SD (標準差)  %Study Variation (%研究變異)  %Tolerance (%公差)
0   Total Gage R&R    0.1216                    7.9834            2.4313
1    Repeatability    0.1216                    7.9834            2.4313
2  Reproducibility    0.0000                    0.0000            0.0000
3     Part-to-Part    1.5178                   99.6808           30.3566
4  Total Variation    1.5227                  100.0000           30.4538

可區分的類別數 (NDC): 17.61, 向下取整數 17.
